In [6]:
!pip install tensorflow==1.0.0
#import sugartensor as tf
#import numpy as np
#import csv
#import string

ERROR: Could not find a version that satisfies the requirement tensorflow==1.0.0
ERROR: No matching distribution found for tensorflow==1.0.0


In [2]:
index2byte = ['<EMP>',' ', 'ṃ','-','ì','y',"'",'Ò',
 'ñ',
 '·',
 'b',
 'q',
 'Y',
 'N',
 'ê',
 '&',
 'ồ',
 'C',
 'á',
 'ú',
 'Æ',
 '.',
 'Ż',
 'L',
 'M',
 'D',
 'û',
 'ó',
 'е',
 'Ñ',
 'ə',
 'ı',
 'ř',
 'â',
 '–',
 'ß',
 'ë',
 'g',
 'ş',
 'Ú',
 '=',
 'ć',
 'e',
 'ø',
 'r',
 'a',
 ':',
 'j',
 'Š',
 'Ϙ',
 'Ş',
 '?',
 'l',
 'S',
 '¿',
 'à',
 'I',
 'B',
 'Á',
 'U',
 '`',
 'H',
 'z',
 'n',
 'k',
 'ï',
 'å',
 'o',
 'Ö',
 'é',
 'ô',
 '¨',
 'Ō',
 'ŏ',
 'š',
 'p',
 'ń',
 'O',
 't',
 '→',
 '…',
 'Ł',
 'P',
 '‘',
 'ý',
 'J',
 'Í',
 'Q',
 'ł',
 'A',
 'G',
 'Ž',
 'ū',
 'Ó',
 'm',
 'Ш',
 'X',
 'F',
 '_',
 '!',
 'f',
 'Ð',
 ',',
 'w',
 'v',
 'Î',
 'h',
 'í',
 'ü',
 'ё',
 'i',
 'ā',
 'ź',
 'æ',
 'ð',
 '￼',
 'Č',
 'ō',
 'u',
 'ç','d','c','x','E','ã','R','K','W','毵','É','ę','Þ','ò','ő','s','T','´','Z','ª','À','ä','č','Œ','œ','V','Ä','ě','¡','ö','ė','ž']

In [3]:
byte2index = {}
for i, ch in enumerate(index2byte):
    byte2index[ch] = i

In [4]:
voca_size = len(index2byte)

In [5]:
def str2index(str_):

    # clean white space
    str_ = ' '.join(str_.split())
    # remove punctuation and make lower case
    str_ = str_.translate(None, string.punctuation).lower()

    res = []
    for ch in str_:
        try:
            res.append(byte2index[ch])
        except KeyError:
            # drop OOV
            pass
    return res

In [6]:
def index2str(index_list):
    # transform label index to character
    str_ = ''
    for ch in index_list:
        if ch > 0:
            str_ += index2byte[ch]
        elif ch == 0:  # <EOS>
            break
    return str_

In [7]:
def print_index(indices):
    for index_list in indices:
        print(index2str(index_list))

In [8]:
@tf.sg_producer_func
def _load_mfcc(src_list):

    # label, wave_file
    label, mfcc_file = src_list

    # decode string to integer
    label = np.frombuffer(label, np.int)

    # load mfcc
    mfcc = np.load(mfcc_file, allow_pickle=False)

    # speed perturbation augmenting
    mfcc = _augment_speech(mfcc)

    return label, mfcc

In [9]:
def _augment_speech(mfcc):

    # random frequency shift ( == speed perturbation effect on MFCC )
    r = np.random.randint(-2, 2)

    # shifting mfcc
    mfcc = np.roll(mfcc, r, axis=0)

    # zero padding
    if r > 0:
        mfcc[:r, :] = 0
    elif r < 0:
        mfcc[r:, :] = 0

    return mfcc

In [10]:
class SpeechCorpus(object):

    def __init__(self, batch_size=16, set_name='train2'):

        # load meta file
        label, mfcc_file = [], []
        with open('/home/kazihar/Converted0/%s.csv' % set_name) as csv_file:
            reader = csv.reader(csv_file, delimiter='\t')
            for row in reader:
                # mfcc file
                mfcc_file.append('/home/kazihar/Converted0/' + row[0])
                # label info ( convert to string object for variable-length support )
                a = row[1].split(',')
                l = []
                for nums in a:
                    m = ""
                    for lets in nums:
                        if lets.isdigit():
                            m = m+lets
                    l.append(int(m))
                label.append(np.asarray(l, dtype=np.int).tostring())

        # to constant tensor
        label_t = tf.convert_to_tensor(label)
        mfcc_file_t = tf.convert_to_tensor(mfcc_file)

        # create queue from constant tensor
        label_q, mfcc_file_q \
            = tf.train.slice_input_producer([label_t, mfcc_file_t], shuffle=True)

        # create label, mfcc queue
        label_q, mfcc_q = _load_mfcc(source=[label_q, mfcc_file_q],
                                     dtypes=[tf.sg_intx, tf.sg_floatx],
                                     capacity=256, num_threads=64)

        # create batch queue with dynamic pad
        batch_queue = tf.train.batch([label_q, mfcc_q], batch_size,
                                     shapes=[(None,), (20, None)],
                                     num_threads=64, capacity=batch_size*32,
                                     dynamic_pad=True)

        # split data
        self.label, self.mfcc = batch_queue
        # batch * time * dim
        self.mfcc = self.mfcc.sg_transpose(perm=[0, 2, 1])
        # calc total batch count
        self.num_batch = len(label) // batch_size

        # print info
        tf.sg_info('%s set loaded.(total data=%d, total batch=%d)'
                   % (set_name.upper(), len(label), self.num_batch))

In [11]:
num_blocks = 3     # dilated blocks
num_dim = 128      # latent dimension

In [12]:
def get_logit(x, voca_size):

    # residual block
    def res_block(tensor, size, rate, block, dim=num_dim):

        with tf.sg_context(name='block_%d_%d' % (block, rate)):

            # filter convolution
            conv_filter = tensor.sg_aconv1d(size=size, rate=rate, act='tanh', bn=True, name='conv_filter')

            # gate convolution
            conv_gate = tensor.sg_aconv1d(size=size, rate=rate,  act='sigmoid', bn=True, name='conv_gate')

            # output by gate multiplying
            out = conv_filter * conv_gate

            # final output
            out = out.sg_conv1d(size=1, dim=dim, act='tanh', bn=True, name='conv_out')

            # residual and skip output
            return out + tensor, out

    # expand dimension
    with tf.sg_context(name='front'):
        z = x.sg_conv1d(size=1, dim=num_dim, act='tanh', bn=True, name='conv_in')

    # dilated conv block loop
    skip = 0  # skip connections
    for i in range(num_blocks):
        for r in [1, 2, 4, 8, 16]:
            z, s = res_block(z, size=7, rate=r, block=i)
            skip += s

    # final logit layers
    with tf.sg_context(name='logit'):
        logit = (skip
                 .sg_conv1d(size=1, act='tanh', bn=True, name='conv_1')
                 .sg_conv1d(size=1, dim=voca_size, name='conv_2'))

    return logit


In [13]:
tf.sg_verbosity(10)

In [14]:
batch_size = 16 

In [15]:
data = SpeechCorpus(batch_size=batch_size * tf.sg_gpus())

I 0610:20:12:59.954:<ipython-input-10-5c53eb943aa3>:50] TRAIN2 set loaded.(total data=89178, total batch=5573)


In [ ]:
x, y = [], []
with open('/home/kazihar/Converted0/train1.csv') as csv_file:
            reader = csv.reader(csv_file, delimiter='\t')
            for row in reader:
                x.append(row[0])
                y.append(str2index(row[1]))
                

In [ ]:
with open('/home/kazihar/Converted0/train2.csv', 'w') as csv_file:
            writer= csv.writer(csv_file, delimiter='\t')
            for i in range(len(x)):
                writer.writerow([x[i], y[i]])

In [16]:
inputs = tf.split(data.mfcc, tf.sg_gpus(), axis=0)

In [17]:
labels = tf.split(data.label, tf.sg_gpus(), axis=0)

In [18]:
seq_len = []
for input_ in inputs:
    seq_len.append(tf.not_equal(input_.sg_sum(axis=2), 0.).sg_int().sg_sum(axis=1))

In [19]:
@tf.sg_parallel
def get_loss(opt):
    # encode audio feature
    logit = get_logit(opt.input[opt.gpu_index], voca_size=voca_size)
    # CTC loss
    return logit.sg_ctc(target=opt.target[opt.gpu_index], seq_len=opt.seq_len[opt.gpu_index])


In [ ]:
tf.sg_train(lr=0.0001, loss=get_loss(input=inputs, target=labels, seq_len=seq_len),
            ep_size=data.num_batch, max_ep=50)

INFO:tensorflow:global_step/sec: 0


I 0610:20:13:20.141:sg_train.py:327] Training started from epoch[000]-step[0].
train:   0%|                       | 2/5573 [00:53<39:12:59, 25.34s/b]

INFO:tensorflow:global_step/sec: 0.0334968


train:   0%|                       | 4/5573 [01:41<36:26:53, 23.56s/b]

INFO:tensorflow:global_step/sec: 0.0333329


...

train:   0%|                      | 21/5573 [07:56<27:46:39, 18.01s/b]

INFO:tensorflow:global_step/sec: 0.0666668


train:   0%|                      | 23/5573 [08:52<34:24:24, 22.32s/b]

INFO:tensorflow:global_step/sec: 0.0333327


train:   1%|▏                     | 62/5573 [23:59<33:33:12, 21.92s/b]

#Recognise

In [8]:
!pip install --ignore-installed librosa 

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
Defaulting to user installation because normal site-packages is not writeable
Processing /home/kazihar/.cache/pip/wheels/6a/ae/a3/9926619fb71320153272a879bcae86f4b1e813b5da5e482a80/librosa-0.7.2-py2-none-any.whl
     |████████████████████████████████| 17.0 MB 649 kB/s eta 0:00:01
  Using cached scikit_learn-0.20.4-cp27-cp27mu-manylinux1_x86_64.whl (5.5 MB)
  Using cached SoundFile-0.10.3.post1-py2.py3-none-any.whl (21 kB)
  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
Processing /home/kazihar/.cache/pip/wheels/fd/b3/9d/2a68163f1c28a0f2d7981a5505e301917559c68b10468bef88/audioread-2.1.9-py2-non

In [ ]:
tf.sg_verbosity(10)

#
# hyper parameters
#

batch_size = 1     # batch size

#
# inputs
#

# vocabulary size
voca_size = data.voca_size

# mfcc feature of audio
x = tf.placeholder(dtype=tf.sg_floatx, shape=(batch_size, None, 20))

# sequence length except zero-padding
seq_len = tf.not_equal(x.sg_sum(axis=2), 0.).sg_int().sg_sum(axis=1)

# encode audio feature
logit = get_logit(x, voca_size=voca_size)

# ctc decoding
decoded, _ = tf.nn.ctc_beam_search_decoder(logit.sg_transpose(perm=[1, 0, 2]), seq_len, merge_repeated=False)

# to dense tensor
y = tf.sparse_to_dense(decoded[0].indices, decoded[0].dense_shape, decoded[0].values) + 1

#
# regcognize wave file
#

# command line argument for input wave file path
tf.sg_arg_def(file=('', 'speech wave file to recognize.'))

# load wave file
wav, _ = librosa.load(tf.sg_arg().file, mono=True, sr=16000)
# get mfcc feature
mfcc = np.transpose(np.expand_dims(librosa.feature.mfcc(wav, 16000), axis=0), [0, 2, 1])

# run network
with tf.Session() as sess:

    # init variables
    tf.sg_init(sess)

    # restore parameters
    saver = tf.train.Saver()
    saver.restore(sess, tf.train.latest_checkpoint('asset/train'))
    # run session
    label = sess.run(y, feed_dict={x: mfcc})

    # print label
    data.print_index(label)